In [1]:
!pip install web3 -q
!pip install pandas -q

In [33]:
import pandas as pd
from web3 import Web3
from web3.middleware import geth_poa_middleware
from multiprocessing import Pool, Manager

In [34]:
def preprocessing(datafile):
    df = pd.read_csv(datafile)
    block_heigth = df["block_number_remove"].max()
    address_df = df.\
        sort_values(["block_number_remove"]).\
        drop_duplicates(subset=["from"], keep="last").\
        reset_index(drop=True)\
        [["from", "block_number_remove"]].\
        rename({"from":"address", "block_number_remove":"use_untill"}, axis="columns")
    return address_df

{
 'accessList': [],
 'blockHash': HexBytes('0xa0638dc5193f66d83cb334896169d00f0921aafb93548d8236ed41eb2090202e'),
 'blockNumber': 14339974,
 'chainId': '0x1',
 'contractAddress': None,
 'cumulativeGasUsed': 30107260,
 'effectiveGasPrice': 22440056776,
 'from': '0x8A4e93F3ef06F5d66a38e5D249Ff22A5EC88700c',
 'gas': 21000,
 'gasPrice': 22440056776,
 'gasUsed': 21000,
 'hash': HexBytes('0x159f39270fc39a1d72e342bc245b6884b21316d190a7dbffde42e2a4cfeedf7f'),
 'input': '0x',
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'maxFeePerGas': 28093259369,
 'maxPriorityFeePerGas': 1500000000,
 'nonce': 0,
 'r': HexBytes('0xe0b4ba8bd1bfea0c9506ca882046f79a24447546c2030145686b677cf5950757'),
 's': HexBytes('0x0b0979bb7cb867ea354c7550ba82527454ee921e799cdad9d6756aeac5150d77'),
 'status': 1,
 'to': '0x25D04117187DA533697e9d8A929C2A5f4aDeBf31',
 'transactionHash': HexBytes('0x159f39270fc39a1d72e342bc245b6884b21316d190a7dbffde42e2a4cfeedf7f'),
 'transactionIndex': 233,
 'type': '0x2',
 'v': 0,
 'value': 200316774637993000
 }

 {
 
 'contractAddress': None,
 'cumulativeGasUsed': 30107260,
 'effectiveGasPrice': 22440056776,
 'gas': 21000,
 'gasPrice': 22440056776,
 'gasUsed': 21000,
 'logs': [],
 
 'maxFeePerGas': 28093259369,
 'maxPriorityFeePerGas': 1500000000,
 'nonce': 0,
 'r': HexBytes('0xe0b4ba8bd1bfea0c9506ca882046f79a24447546c2030145686b677cf5950757'),
 's': HexBytes('0x0b0979bb7cb867ea354c7550ba82527454ee921e799cdad9d6756aeac5150d77'),
 'v': 0,
 }

In [4]:
def single(address_df, edgefile, nodefile, w3):
    max_block_heigth = address_df["use_untill"].max()
    address_set = set(address_df["address"].values)
    with open(edgefile, "w", encoding="UTF8") as tx_file:
        tx_file.write("from,to,value,gas,hash,input,blockNumber,transactionIndex\n")
        for block_number in range(max_block_heigth, -1, -1):
            block = w3.eth.get_block(block_number)
            for transaction in block.transactions[::-1]:
                tx = {**w3.eth.get_transaction(transaction.hex()), **w3.eth.get_transaction_receipt(transaction.hex())}
                if tx["status"] == 1 and tx["to"] in address_set and tx["to"] in address_df[address_df["use_untill"]<=block_number]["address"].values:
                    tx_file.write("{},{},{},{},{},{},{},{}\n".format(
                        tx["from"], tx["to"], tx["value"], tx["gas"], tx["hash"].hex(), tx["input"][:10], tx["blockNumber"], tx["transactionIndex"]
                    ))
                    if tx["from"] not in address_set:
                        address_set.add(tx["from"])
                        row = pd.DataFrame.from_dict({"address": [tx["from"]], "use_untill": [block_number]})
                        address_df = pd.concat([address_df, row], ignore_index=True)
    address_df.to_csv(nodefile, index=False)
    return

In [43]:
def multi(address_df, edgefile, nodefile, w3, depth=100, mode="w"): #depth compreso
    def task(start, lock):
        new_level_address_subset = set()
        new_level_address_subdf = pd.DataFrame.from_dict({"address": [], "use_untill": [], "level": []})
        for block_number in range(min(start+step-1, max_block_heigth), start-1, -1):
            block = w3.eth.get_block(block_number)
            for transaction in block.transactions[::-1]:
                tx = {**w3.eth.get_transaction(transaction.hex()), **w3.eth.get_transaction_receipt(transaction.hex())}
                if tx["status"] == 1 and tx["to"] in curr_level_address_set and tx["to"] in curr_level_address_df[curr_level_address_df["use_untill"]<=block_number]["address"].values:
                    with lock:
                        tx_file.write("{},{},{},{},{},{},{},{}\n".format(
                            tx["from"], tx["to"], tx["value"], tx["gas"], tx["hash"].hex(), tx["input"][:10], tx["blockNumber"], tx["transactionIndex"], new_level
                        ))
                    if tx["from"] not in new_level_address_subset:
                        new_level_address_subset.add(tx["from"])
                        row = pd.DataFrame.from_dict({"address": [tx["from"]], "use_untill": [block_number], "level": [new_level]})
                        new_level_address_subdf = pd.concat([new_level_address_subdf, row], ignore_index=True)
        return new_level_address_subdf
    if mode == "w":
        if "level" not in address_df.columns:
            address_df["level"] = 0
        address_df.to_csv(nodefile, index=False)
        with open(edgefile, "w", encoding="UTF8") as tx_file:
            tx_file.write("from,to,value,gas,hash,input,blockNumber,transactionIndex,level\n")
        max_block_heigth = address_df["use_untill"].max()
        step = 1000
        curr_level = 0
        curr_level_address_set = set(address_df["address"].values)
        curr_level_address_df = address_df
        while curr_level < depth:
            new_level = curr_level + 1
            #generate new level of edges and nodes
            with Manager() as manager:
                lock = manager.Lock()
                with Pool() as pool:
                    items = [(i, lock) for i in range(0, max_block_heigth+1, step)]
                    new_level_address_subdf_list = pool.starmap_async(task, items)
                    new_level_address_subdf_list.wait()
            new_level_address_df = pd.concat(new_level_address_subdf_list.get())
            #elimina da new_level_address_df tutte le righe con address ripeturi e con use_untill che non è massimo tra i doppioni
            #elimina da new_level_address_df le righe che hanno address presenti in address_df
            address_df = pd.concat([new_level_address_df])
            new_level_address_df.to_csv(nodefile, mode="a", header=False, index=False)
            curr_level += 1
            curr_level_address_set = set(new_level_address_df["address"].values)
            curr_level_address_df = new_level_address_df
    return curr_level_address_df

In [5]:
eth_url = 'https://mainnet.infura.io/v3/e0a4e987f3ff4f4fa9aa21bb08f09ef5'
bsc_url = "https://bsc-dataseed.binance.org/"

eth_datafile = 'one_day_exit_scam_eth.csv'
bsc_datafile = 'one_day_exit_scam_bsc.csv'

eth_edgefile = 'tx_eth.csv'
bsc_edgefile = 'tx_bsc.csv'

eth_nodefile = 'address_eth.csv'
bsc_nodefile = 'address_bsc.csv'

eth_w3 = Web3(Web3.HTTPProvider(eth_url))
bsc_w3 = Web3(Web3.HTTPProvider(bsc_url))
bsc_w3.middleware_onion.inject(geth_poa_middleware, layer=0)

eth_address_df = preprocessing(eth_datafile)
bsc_address_df = preprocessing(bsc_datafile)

In [39]:
single(eth_address_df, eth_edgefile, eth_nodefile, eth_w3)

{'accessList': [],
 'blockHash': HexBytes('0xa0638dc5193f66d83cb334896169d00f0921aafb93548d8236ed41eb2090202e'),
 'blockNumber': 14339974,
 'chainId': '0x1',
 'contractAddress': None,
 'cumulativeGasUsed': 30107260,
 'effectiveGasPrice': 22440056776,
 'from': '0x8A4e93F3ef06F5d66a38e5D249Ff22A5EC88700c',
 'gas': 21000,
 'gasPrice': 22440056776,
 'gasUsed': 21000,
 'hash': HexBytes('0x159f39270fc39a1d72e342bc245b6884b21316d190a7dbffde42e2a4cfeedf7f'),
 'input': '0x',
 'logs': [],
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
single(bsc_address_df, bsc_edgefile, bsc_nodefile, bsc_w3)

In [ ]:
multi(eth_address_df, eth_edgefile, eth_nodefile, eth_w3)

In [ ]:
multi(bsc_address_df, eth_edgefile, eth_nodefile, eth_w3)

In [48]:
def s(ide, lock):
    with lock:
        print(ide)
    return (ide, ide)
with Manager() as manager:
    # create the shared lock
    lock = manager.Lock()
    # create and configure the process pool
    with Pool() as pool:
        # prepare task arguments
        items = [(i, lock) for i in range(10)]
        # issue tasks into the process pool
        result = pool.starmap_async(s, items)
        # wait for all tasks to finish
        result.wait()
result.get()


0
1
2
4
5
3
6
7
8
9


[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9)]

In [35]:
q = pd.DataFrame({"l":[]})

In [42]:
"l" in q.columns

True